# sim plan

* for every round
  * procedure?
        * if yes: 
            * offer to random resident (and choose intern who would have gotten it either way for log)
                * if accepts: increment and log to both
                * else: log miss only
        * else
            * log missed procedure0
    * 

# first attempt with no probability:

In [74]:
from collections import namedtuple

import numpy as np
import pandas as pd
from icecream import ic

In [75]:
# custom datatype for procedure offer logs
Procedure_Outcome = namedtuple(
    "Procedure_Outcome",
    [
        "day",
        "senior",
        "senior_prior_procedures",
        "intern",
        "intern_prior_procedures",
        "nth_procedure_opportunity_of_the_day",
        "performed",
    ],
)

In [76]:
residents = pd.read_excel("residents.xlsx")
residents

,name,training_level,paracentesis_completed
0,R2 1,senior,0
1,R2 2,senior,0
2,R2 3,senior,0
3,R2 4,senior,0
4,R2 5,senior,0
5,R2 6,senior,0
6,R2 7,senior,0
7,R2 8,senior,0
8,R2 9,senior,0
9,R2 10,senior,0


In [77]:
seniors = residents.training_level == "senior"
interns = residents.training_level == "intern"

In [78]:
days = 30
signoff_threshold = 1
signed_off_avidity = 0.8
not_signed_off_avidity = 0.1

outcome_log = list()


for day in range(days):
    # TODO: determine number offered (poisson)
    procedure_opportunity_today = 2  # artificial for testing
    for procedure_opportunity in range(procedure_opportunity_today):

        selected_senior = residents[seniors].sample().name.values[0]
        selected_intern = residents[interns].sample().name.values[0]

        if (
            residents.loc[
                residents.name.isin([selected_senior, selected_intern]),
                "paracentesis_completed",
            ]
            >= signoff_threshold
        ).any():
            avidity = signed_off_avidity
        else:
            avidity = not_signed_off_avidity

        performed = None

        if np.random.rand() <= avidity:
            senior_prior_procedures = residents.loc[
                residents.name == selected_senior, "paracentesis_completed"
            ].values[0]

            residents.loc[
                residents.name == selected_senior, "paracentesis_completed"
            ] += 1

            intern_prior_procedures = residents.loc[
                residents.name == selected_intern, "paracentesis_completed"
            ].values[0]

            residents.loc[
                residents.name == selected_intern, "paracentesis_completed"
            ] += 1
            performed = True
        else:
            performed = False

        today_outcome = Procedure_Outcome(
            day=day,
            senior=selected_senior,
            senior_prior_procedures=senior_prior_procedures,
            intern=selected_intern,
            intern_prior_procedures=intern_prior_procedures,
            nth_procedure_opportunity_of_the_day=procedure_opportunity,
            performed=performed,
        )
        outcome_log.append(today_outcome)

In [79]:
outcomes = pd.DataFrame(outcome_log)
outcomes

,day,senior,senior_prior_procedures,intern,intern_prior_procedures,nth_procedure_opportunity_of_the_day,performed
0,0,R2 3,5,R1 9,13,0,False
1,0,R2 7,5,R1 8,13,1,False
2,1,R3 1,5,R1 5,0,0,True
3,1,R2 4,5,R1 4,0,1,False
4,2,R2 5,5,R1 6,0,0,False
5,2,R3 10,5,R1 4,0,1,False
6,3,R2 5,5,R1 7,0,0,False
7,3,R2 4,0,R1 2,1,1,True
8,4,R3 2,2,R1 7,0,0,True
9,4,R3 8,2,R1 10,0,1,False


In [80]:
outcomes.performed.sum() / len(outcomes)

np.float64(0.5833333333333334)